<a href="https://colab.research.google.com/github/ramzesssina/NLPrespos/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание 2**

In [61]:
!pip install pymorphy3

In [62]:
import pymorphy3
import nltk
from string import punctuation
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from nltk.corpus import stopwords
from nltk import SnowballStemmer
from math import log

In [63]:
texts = ['Лес шумел, лес тянулся вдаль,',
         'В лесу было темно, темно и тихо.',
         'В тени деревьев прятался лес,',
         'Туда, где лес, где свет, где тень.',
         'Лес скрывал всё, лес был всесилен.'
         ]

In [64]:
nltk.download('stopwords')

russian_stopwords = stopwords.words("russian")
morph = pymorphy3.MorphAnalyzer()
stemmer = SnowballStemmer('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Изменили текст применив лемматизацию**

In [65]:
def change_text(texts):
    words = texts.lower().split()
    lemmatized = []

    stop_free = []
    for word in words:
      if word not in russian_stopwords:
        stop_free.append(word)

    punc_free = ""
    for char in " ".join(stop_free):
      if char not in punctuation:
        punc_free += char

    for word in punc_free.split():
        lemma = morph.parse(word)[0].normal_form
        lemmatized.append(lemma)

    return " ".join(lemmatized)

In [66]:
change_text = [change_text(text) for text in texts]
print(change_text)

['лес шуметь лес тянуться вдаль', 'лес темно темно тихо', 'тень дерево прятаться лес', 'туда лес свет тень', 'лес скрывать всё лес всесильный']


# **Применил Токенизацию**

In [67]:
def tokin(text):
    clear_sub = re.sub(r"[^а-яА-ЯёЁ\s]", "", text)
    words = clear_sub.split()
    return words

In [68]:
Tokin_for_lemma = [tokin(text) for text in change_text]
print(Tokin_for_lemma)

[['лес', 'шуметь', 'лес', 'тянуться', 'вдаль'], ['лес', 'темно', 'темно', 'тихо'], ['тень', 'дерево', 'прятаться', 'лес'], ['туда', 'лес', 'свет', 'тень'], ['лес', 'скрывать', 'всё', 'лес', 'всесильный']]


# **Bag Of Words**

In [69]:
unique_words = []
for sublist in Tokin_for_lemma:
  for text in sublist:
    if text not in unique_words:
      unique_words.append(text)
unique_words = sorted(unique_words)
print(unique_words)

['вдаль', 'всесильный', 'всё', 'дерево', 'лес', 'прятаться', 'свет', 'скрывать', 'темно', 'тень', 'тихо', 'туда', 'тянуться', 'шуметь']


In [70]:
def bag_of_words(Tokin_for_lemma, unique_words):
    matrix = []
    for sublist in Tokin_for_lemma:
        vector = [sublist.count(word) for word in unique_words]
        matrix.append(vector)
    return np.array(matrix)

bow_matrix = bag_of_words(Tokin_for_lemma, unique_words)
print(unique_words)
bow_matrix

['вдаль', 'всесильный', 'всё', 'дерево', 'лес', 'прятаться', 'свет', 'скрывать', 'темно', 'тень', 'тихо', 'туда', 'тянуться', 'шуметь']


array([[1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

# **TF-IDF**

In [71]:
def manual_tfidf(bow_matrix):
    tf = bow_matrix / bow_matrix.sum(axis=1, keepdims=True)

    idf = np.array([log(len(bow_matrix) / (1 + np.count_nonzero(bow_matrix[:, i]))) for i in range(bow_matrix.shape[1])])

    tfidf = tf * idf
    return tfidf

tfidf_matrix = manual_tfidf(bow_matrix)
print(unique_words)
print("Матрица TF-IDF:")
print(tfidf_matrix)

['вдаль', 'всесильный', 'всё', 'дерево', 'лес', 'прятаться', 'свет', 'скрывать', 'темно', 'тень', 'тихо', 'туда', 'тянуться', 'шуметь']
Матрица TF-IDF:
[[ 0.18325815  0.          0.          0.         -0.07292862  0.
   0.          0.          0.          0.          0.          0.
   0.18325815  0.18325815]
 [ 0.          0.          0.          0.         -0.04558039  0.
   0.          0.          0.45814537  0.          0.22907268  0.
   0.          0.        ]
 [ 0.          0.          0.          0.22907268 -0.04558039  0.22907268
   0.          0.          0.          0.12770641  0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.04558039  0.
   0.22907268  0.          0.          0.12770641  0.          0.22907268
   0.          0.        ]
 [ 0.          0.18325815  0.18325815  0.         -0.07292862  0.
   0.          0.18325815  0.          0.          0.          0.
   0.          0.        ]]
